In [3]:
# !pip install cohere -q
# !pip install langchain -q
# !pip install langchain-cohere -q
# !pip install -U langchain-community tavily-python -q
# !pip install wikipedia -q
# !pip install langchain_experimental -q
# !pip install langgraph -q

In [4]:
from google.colab import userdata
import os

os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "learning-assistant"

## Create Tools

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results=5)
python_repl_tool = PythonREPLTool()

## Helper Function

In [16]:
from langchain.agents import AgentExecutor
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_cohere import ChatCohere
from langchain_cohere import create_cohere_tools_agent, create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

def create_agent(llm: ChatCohere, tools: list, system_prompt: str):
  prompt = ChatPromptTemplate.from_messages(
      [
          (
              "system",
              system_prompt,
          ),
          MessagesPlaceholder(variable_name="messages"),
          MessagesPlaceholder(variable_name="agent_scratchpad"),
      ]
  )
  agent = create_cohere_react_agent(llm, tools, prompt)
  executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
  return executor


In [17]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    print(result)
    return {"messages": [HumanMessage(content=result["text"], name=name)]}

In [18]:
from typing import Iterable

from langchain_core.messages import AIMessage, AIMessageChunk

def parse(ai_message: AIMessage) -> dict:
    """Parse the AI message."""
    print(ai_message, type(ai_message))
    return {
        "next": ai_message.content
    }

## Agent Supervisor

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser

members = ["Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
options = ["FINISH"] + members

tools = [
    {
        "title": "route",
        "description": "Select the next role",
        "properties": {
            "next": {
                "type": "object",
                "required": True,
                "properties": {
                    "next": {
                      "title": "Next",
                      "anyOf": [
                          {"enum": options},
                      ],
                    }
                },
            }
        }
    }
]
# op = {"next": "FINISH"}
# json_prompt = f" give response in this JSON format: {op}"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}"
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatCohere()

supervisor_chain = (
    prompt
    | llm.bind_tools(tools=tools)
) | parse

## Construct Graph

In [20]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
import operator
import functools
from langgraph.graph import StateGraph, END

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]
  next: str

research_agent = create_agent(llm, [tavily_tool], "You are a web researcher")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

code_agent = create_agent(llm, [python_repl_tool], "You may generate safe python code to analyze data and generate charts using matplotlib.")
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)

In [21]:
for member in members:
  workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [22]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Do research about latest lok sabha elections in India and present the result in graph.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

content='Researcher' additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'da08a6a3-584f-4e2a-ba6e-7b8e0534e304', 'token_count': {'input_tokens': 176, 'output_tokens': 2}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'da08a6a3-584f-4e2a-ba6e-7b8e0534e304', 'token_count': {'input_tokens': 176, 'output_tokens': 2}} id='run-77f614ef-ef3f-45c3-8caa-3c1e192620bb-0' <class 'langchain_core.messages.ai.AIMessage'>
{'supervisor': {'next': 'Researcher'}}
----


> Entering new AgentExecutor chain...

I will search for the latest Lok Sabha elections in India and then present the results in a graph.
{'tool_name': 'tavily_search_results_json', 'parameters': {'query': 'latest lok sabha elections in india'}}
[{'url': 'https://indianexpress.com/elections/lok-sabha-election-2024-voting-live-updates-phase-

KeyError: 'text'

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper

search = TavilySearchAPIWrapper(tavily_api_key=tavily_api_key)
tavily_internet_search = TavilySearchResults(api_wrapper=search)

In [ ]:
"""Tool for the Wikipedia API."""

from typing import Optional

from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.tools import BaseTool

from langchain_community.utilities.wikipedia import WikipediaAPIWrapper


class WikipediaQueryLoad(BaseTool):
    """Tool that searches the Wikipedia API."""

    name: str = "wikipedia"
    description: str = (
        "A wrapper around Wikipedia. "
        "Useful for when you need to answer general questions about "
        "people, places, companies, facts, historical events, or other subjects. "
        "Input should be a search query."
    )
    api_wrapper: WikipediaAPIWrapper

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Wikipedia tool."""
        print(query)
        return self.api_wrapper.load(query)


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)
wiki_tool = WikipediaQueryLoad(api_wrapper=api_wrapper)

In [ ]:
internet_search = wiki_tool

In [ ]:
from langchain.agents import AgentExecutor
from langchain_cohere.chat_models import ChatCohere
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

llm = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus")
agent = create_cohere_react_agent(
    llm=llm,
    tools=[internet_search],
    # prompt=chat_template,
    prompt=ChatPromptTemplate.from_template("{input}"),
)
agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)

In [ ]:
preamble = """
You are an expert in preparing comprehensive learning material for the user input.
"""
response = agent_executor.invoke(
    {
        "input": "Large Lanugage Model",
        "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will search for 'Large Language Model' and write an answer based on the information I find.
{'tool_name': 'wikipedia', 'parameters': {'query': 'Large Language Model'}}
Large Language Model
[Document(page_content='A large language model (LLM) is a language model notable for its ability to achieve general-purpose ', metadata={'title': 'Large language model', 'summary': 'A large language model (LLM) is a language model notable for its ability to achieve general-purpose language generation and other natural language processing tasks such as classification. LLMs acquire these abilities by learning statistical relationships from text documents during a computationally intensive self-supervised and semi-supervised training process. LLMs can be used for text generation, a form of generative AI, by taking an input text and repeatedly predicting the next token or word.\nLLMs are artificial neural networks. The largest and most capable, as of March 2024,

In [ ]:
print(response["output"])

A large language model (LLM) is a language model notable for its ability to achieve general-purpose language generation and other natural language processing tasks such as classification. LLMs are artificial neural networks. The largest and most capable, as of March 2024, are built with a decoder-only transformer-based architecture. 

LLMs are trained using a computationally intensive self-supervised and semi-supervised training process, where they learn statistical relationships from text documents. This allows them to be used for text generation, a form of generative AI, by taking an input text and repeatedly predicting the next token or word. 

Some notable LLMs include: 
- OpenAI's GPT series of models (e.g., GPT-3.5 and GPT-4, used in ChatGPT and Microsoft Copilot)
- Google's PaLM and Gemini (the latter of which is currently used in the chatbot of the same name)
- xAI's Grok
- Meta's LLaMA family of models
- Anthropic's Claude models
- Mistral AI's models
- Databricks' DBRX

LLMs 